# Running periodic ground-state structure solver
In this example, we illustrate how to create a periodic ground-state solver from a cluster expansion and how to obtain a ground-state upper-bound estimation. This implementation solves for the ground state for a fixed super-cell size stored as the supercell matrix in the ensemble's processor. Meaning that the solution corresponds only to an upper bound of the global ground state of the corresponding infinitely sized system.

cvxpy and mixed integer programming solvers (such as SCIP, GUROBI, CPLEX) are required. Details of their installation can be found in: https://www.cvxpy.org/tutorial/advanced/index.html#choosing-a-solver

For more details on the nature of the global problem, finding upper and lower bounds please see:
https://doi.org/10.1103/PhysRevB.94.134424

In [1]:
import copy

import numpy as np
import matplotlib.pyplot as plt
from monty.serialization import loadfn, dumpfn
from pymatgen.analysis.ewald import EwaldSummation
from pymatgen.core import Structure, Lattice

from smol.cofe import ClusterSubspace, ClusterExpansion
from smol.cofe.extern import EwaldTerm
from smol.cofe.space.domain import get_allowed_species

from smol.moca import Ensemble

from smol.capp.generate import PeriodicGroundStateSolver

### 0) Create a Cluster Subspace based on the disordered structure with an Ewald term

In [2]:
# Hypothetical quarternary system
clof = Structure(Lattice.cubic(2.0),
                 [{"Ca2+": 0.5, "Li+":0.5}, {"O2-": 0.5, "F-":0.5}],
                 [[0,0,0], [0.5, 0.5, 0.5]])

In [3]:
subspace = ClusterSubspace.from_cutoffs(structure=clof, 
                                        cutoffs={2:4.0, 3:3.0})

subspace.add_external_term(EwaldTerm(eta=None)) # Add the external Ewald Term

In [4]:
print("Species:", get_allowed_species(clof))

Species: [[Species Li+, Species Ca2+], [Species O2-, Species F-]]


In [5]:
print("Number of correlation functions (wo. Ewald):", subspace.num_corr_functions)

Number of correlation functions (wo. Ewald): 21


In [6]:
print("Number of orbits:", subspace.num_orbits)

Number of orbits: 21


### 1) Create random ECIs and ClusterExpansion

In [7]:
coefs = np.random.random(size=subspace.num_corr_functions+1)
coefs[0] = -10
coefs[-1] = 0.1

In [8]:
ce = ClusterExpansion(subspace, coefs)

### 2) Create a semi-grand canonical ensemble
By default, the Ensemble object uses ClusterDecompositionProcessor, which would greatly reduce the amount of many-body terms in pseudo-Boolean function. Switching to ClusterExpansionProcessor is allowed but not recommended.

The upper-bound supercell size is defined within the Ensemble. Here, we use a supercell containing 2 primitive units. A semigrand-canonical ensemble is defined as an Ensemble object with chemical potentials specified.

In this example, we used a relatively simple chemical space and a small supercell for a quick demonstration. The Boolean problem size grows exponentially with increasing composition complexity, therefore, the solver might not be able to solve the upper-bound in a large super-cell with high configurational entropy. 

In [9]:
chempots = {"Li+": 0.0, "Ca2+": 0.0, "O2-":0.0, "F-": 0.0}
grand_ens = Ensemble.from_cluster_expansion(ce, np.diag([2, 2, 2]),
                                            chemical_potentials=chempots)

### 3) Create a solver instance from the ensemble
Charge-balance constraints are included by default. If any other constraint is needed, refer to the documentation of GroundStateSolver.

The default solver is "SCIP". For other solvers supported by cvxpy, see: https://www.cvxpy.org/tutorial/advanced/index.html#setting-solver-options

In [10]:
# Use 1e-6 as a cutoff to cluster terms. Any term with coefficient lower than 1e-6 will not be included into the optimization.
grand_solver = PeriodicGroundStateSolver(grand_ens, term_coefficients_cutoff=1e-6)
print("Number of variables:", grand_solver._canonicals.variables.size)
print("Number of auxiliary variables:", grand_solver._canonicals.auxiliary_variables.size)
print("Number of constraints:", len(grand_solver._canonicals.constraints))

Number of variables: 32
Number of auxiliary variables: 960
Number of constraints: 3345


### 4) Solve the problem in semi-grand canonical ensemble

In [11]:
grand_solver.solve()
energy = grand_solver.ground_state_energy
structure = grand_solver.ground_state_structure

print("Ground-state energy, un-normalized(eV):", energy)
print("Ground-state structure:", structure)

Ground-state energy, un-normalized(eV): -105.66851715273324
Ground-state structure: Full Formula (Ca8 O8)
Reduced Formula: CaO
abc   :   4.000000   4.000000   4.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (16)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Ca2+  0     0     0
  1  Ca2+  0     0     0.5
  2  Ca2+  0     0.5   0
  3  Ca2+  0     0.5   0.5
  4  Ca2+  0.5   0     0
  5  Ca2+  0.5   0     0.5
  6  Ca2+  0.5   0.5   0
  7  Ca2+  0.5   0.5   0.5
  8  O2-   0.25  0.25  0.25
  9  O2-   0.25  0.25  0.75
 10  O2-   0.25  0.75  0.25
 11  O2-   0.25  0.75  0.75
 12  O2-   0.75  0.25  0.25
 13  O2-   0.75  0.25  0.75
 14  O2-   0.75  0.75  0.25
 15  O2-   0.75  0.75  0.75


### 5) Create and solve a canonical ensemble problem
Our solver also supports solving the ground-state in canonical ensembles. In doing so, one only needs to create a canonical ensemble (i.e., an ensemble with no chemical potentials provided). When using a canonical ensemble, note that either a fixed composition or an intial occupancy to determine the composition to fix must be provided as an argument to initialize the solver.

In [12]:
# Creating the ensemble.
canonical_ens = Ensemble.from_cluster_expansion(ce, np.diag([2, 2, 2]))
print("Sublattices:", canonical_ens.sublattices)
print("Bits:", [s.species for s in canonical_ens.sublattices])

Sublattices: [Sublattice(site_space=Ca2+0.5 Li+0.5 , sites=array([0, 1, 2, 3, 4, 5, 6, 7]), active_sites=array([0, 1, 2, 3, 4, 5, 6, 7]), encoding=array([0, 1])), Sublattice(site_space=O2-0.5 F-0.5 , sites=array([ 8,  9, 10, 11, 12, 13, 14, 15]), active_sites=array([ 8,  9, 10, 11, 12, 13, 14, 15]), encoding=array([0, 1]))]
Bits: [(Species Li+, Species Ca2+), (Species O2-, Species F-)]


In [13]:
# Fix to Ca4 Li4 O4 F4.
canonical_solver = PeriodicGroundStateSolver(
    canonical_ens,
    term_coefficients_cutoff=1e-6,
    initial_occupancy=[0, 1, 1, 0, 1, 0, 1, 0,
                       1, 0, 1, 0, 0, 1, 0, 1])
print("Number of variables:", canonical_solver._canonicals.variables.size)
print("Number of auxiliary variables:", canonical_solver._canonicals.auxiliary_variables.size)
print("Number of constraints:", len(canonical_solver._canonicals.constraints))

Number of variables: 32
Number of auxiliary variables: 960
Number of constraints: 3348


In [14]:
canonical_solver.solve()
energy = canonical_solver.ground_state_energy
structure = canonical_solver.ground_state_structure

print("Ground-state energy, un-normalized(eV):", energy)
print("Ground-state structure:", structure)

Ground-state energy, un-normalized(eV): -96.33719852646621
Ground-state structure: Full Formula (Li4 Ca4 O4 F4)
Reduced Formula: LiCaOF
abc   :   4.000000   4.000000   4.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (16)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Li+   0     0     0
  1  Li+   0     0     0.5
  2  Ca2+  0     0.5   0
  3  Ca2+  0     0.5   0.5
  4  Ca2+  0.5   0     0
  5  Ca2+  0.5   0     0.5
  6  Li+   0.5   0.5   0
  7  Li+   0.5   0.5   0.5
  8  O2-   0.25  0.25  0.25
  9  O2-   0.25  0.25  0.75
 10  F-    0.25  0.75  0.25
 11  F-    0.25  0.75  0.75
 12  O2-   0.75  0.25  0.25
 13  O2-   0.75  0.25  0.75
 14  F-    0.75  0.75  0.25
 15  F-    0.75  0.75  0.75
